In [26]:
import skimage
import skimage.io as io
import scipy.ndimage as ndi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn import cluster
from sklearn import metrics
from scipy.optimize import minimize_scalar
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import pairwise_distances

from scipy.spatial.distance import pdist

import optuna
import time

from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 60em; }</style>"))

SCALE = False
TEST_SIZE = 5000

EPS_MIN = 2
EPS_MAX = 2.5
TIMEOUT = 60*10*3

Q = None
EPS = None


ic = io.imread_collection('data/*.png', conserve_memory=False)
print(len(ic))
#io.imshow_collection(ic)

C:\Users\Hyperbook\AppData\Local\Temp\ipykernel_13424\519000195.py:21: DeprecationWarning:

Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display



7618


In [27]:
#For test
correct_df = pd.read_csv('data/@0CLUSTERING.csv')
print(correct_df)

correct = list(correct_df["Cluster"])

#correct

           Filename      Cluster
0       1_18-41.png        1like
1       1_18-96.png  @toolong129
2      1_18-174.png            T
3      1_19-204.png            S
4      1_19-229.png            r
...             ...          ...
7613  7_521-166.png            r
7614  7_521-241.png            S
7615  7_522-189.png            t
7616   7_523-19.png       0PLAMY
7617   7_523-95.png       0PLAMY

[7618 rows x 2 columns]


In [28]:
initial_df = pd.DataFrame(dtype=object)
initial_df[0] = ic
initial_df[1] = correct

initial_df[0] = pd.DataFrame(initial_df.apply(lambda row: skimage.color.rgb2gray(row[0]), axis=1))

In [29]:
def data_prep(df, scale):
    #print(df.apply(lambda row: ndi.sum_labels(row[0]),axis=1))
    max_sum = max(df.apply(lambda row: ndi.sum_labels(row[0]),axis=1))
    #print(max_sum)
    
    def get_max_shape(df):
        shape_y = max(df.apply(lambda row: row[0].shape[0], axis=1))
        shape_x = max(df.apply(lambda row: row[0].shape[1], axis=1))
        return shape_y, shape_x
    
    if scale:
        def img_scale(img):
            my_sum = ndi.sum_labels(img)
            return ndi.zoom(img, np.sqrt(max_sum/my_sum), cval=1.)

        df[0] = df.apply(lambda row: img_scale(row[0]), axis=1)
        #print(df.apply(lambda row: ndi.sum_labels(row[0]),axis=1))

    def img_pre(img):
        cy, cx = ndi.center_of_mass(img)
        cy = round(cy)
        cx = round(cx)
        sy, sx = img.shape
        top = max(sy - 1 - cy - cy, 0)
        bot = max(cy - (sy - 1 - cy), 0)
        left = max(sx - 1 - cx - cx, 0)
        right = max(cx - (sx - 1 - cx), 0)
        return cv2.copyMakeBorder(img, top, bot, left, right, cv2.BORDER_CONSTANT, None, value=1.)

    df[0] = df.apply(lambda row: img_pre(row[0]), axis=1)
    #print(get_max_shape(df))

    max_shape_y, max_shape_x = get_max_shape(df)

    def img_post(img):
        sy, sx = img.shape
        top = (max_shape_y - sy) // 2
        bot = (max_shape_y - sy + 1) // 2
        left = (max_shape_x - sx) // 2
        right = (max_shape_x - sx + 1) // 2
        return cv2.copyMakeBorder(img, top, bot, left, right, cv2.BORDER_CONSTANT, None, value=1.)

    df[0] = df.apply(lambda row: img_post(row[0]), axis=1)
    #print(get_max_shape(df))
    
    df[0] = df.apply(lambda row: np.reshape(row[0], -1), axis=1)

data_prep(initial_df, SCALE)



In [30]:
def get_subsets(df):
    ind = list(df.index)
    np.random.shuffle(ind)
    ind1 = ind[:TEST_SIZE]
    ind2 = ind[len(df) - TEST_SIZE:]
    np.random.shuffle(ind1)
    np.random.shuffle(ind2)
    return [pd.DataFrame(df, index=ind1, copy=False),
            pd.DataFrame(df, index=ind2, copy=False)]

def get_subset(df):
    sub, _ = get_subsets(df)
    return sub

In [31]:
def clustering(eps, q, data):
    print(f"started clustering({eps}, {q})")
    if np.isclose(q, 0):
        p = np.inf
    else:
        p = 1 / q
    
    if np.isclose(p, 2):
        p = 2
    
    start = time.time()
    _, labels = cluster.dbscan(data, eps=eps, p=p, min_samples=1)
    stop = time.time()
    print(f"clustering({eps}, {q}) finished, time: {stop - start}")
    #dbscan.fit(data)
    return labels

In [32]:
def show_plot(data_x, y):
    if not SCALE:
        pca = PCA(n_components = 2)
        data2D = pca.fit_transform(data_x)
        print(f"nr of classes: {len(set(y))}")
        fig = px.scatter(x=data2D[:, 0], y=data2D[:, 1], color=[str(v) for v in y], width=900, height=600)
        fig.show()
    else:
        print(f"{SCALE=}, plot not shown")

In [33]:
def objective(eps, q):
    acc = []
    for test_df in get_subsets(initial_df):
        data = list(test_df[0])
        correct = list(test_df[1])
        res = clustering(eps, q, data)
        acc.append(metrics.adjusted_rand_score(correct, res))
    return np.mean(acc)

def show_objective_plot(up_bound, num):
    x = np.linspace(0.001, up_bound, num=num)
    objective_vec = np.vectorize(objective)
    y = objective_vec(x)
    fig = px.scatter(x=x, y=y, width=900, height=600)
    fig.show()

#show_objective_plot(4, 100)

In [34]:
def check_clustering(eps, q):
    test_df = get_subset(initial_df)
    data = list(test_df[0])
    correct = list(test_df[1])
    res = clustering(eps, q, data)
    acc = metrics.adjusted_rand_score(correct, res)
    
    print("correct clustering")
    show_plot(data, correct)
    print("\n")
    
    print(f"found clustering")
    print(f"eps: {float(eps)}")
    print(f"q: {float(q)}")
    print(f"ACCURACY: {metrics.rand_score(correct, res)}")
    print(f"BALANCE ACCURACY: {acc}")
    
    acc1 = objective(eps, q)
    
    print(f"AVERAGE BALANCED ACCURACY: {np.mean([acc1, acc1, acc])}")
    
    show_plot(data, res)


In [35]:
#test
#%time
#_, labels = cluster.dbscan(list(initial_df.loc[:10, 0]), eps=1, p=2, min_samples=1)


In [36]:
#check_clustering(2.3, 1/3) #test

In [37]:
def show_all_data():
    print("all data")
    show_plot(list(initial_df[0]), list(initial_df[1]))

show_all_data()

all data
nr of classes: 283


In [38]:
def objective_optuna(trial):
    eps = trial.suggest_float("eps", EPS_MIN, EPS_MAX)
    q = trial.suggest_float("q", 0, 1)
    return objective(eps, q)

In [39]:
def get_best_params_optuna(timeout):
    study = optuna.create_study(direction="maximize")
    
    fixed_params = {key: val for key, val in [("q", Q), ("eps", EPS)] if val is not None}
    study.sampler = optuna.samplers.PartialFixedSampler(fixed_params, study.sampler)
    
    study.optimize(objective_optuna, timeout=timeout, gc_after_trial=True)
    optuna.visualization.plot_slice(study).show()
    optuna.visualization.plot_contour(study, params=["q", "eps"]).show()
    optuna.visualization.plot_param_importances(study).show()
    return study.best_trial.params["eps"], study.best_trial.params["q"]

In [40]:
#print(initial_df)
def get_best_params():
    return get_best_params_optuna(TIMEOUT)

best_eps, best_q = get_best_params()


[I 2022-05-04 08:23:05,073] A new study created in memory with name: no-name-d84564e6-2b2a-4ef6-a0f3-0702a54ca1b0
C:\Users\Hyperbook\AppData\Local\Temp\ipykernel_13424\1268617533.py:5: ExperimentalWarning:

PartialFixedSampler is experimental (supported from v2.4.0). The interface can change in the future.



started clustering(2.497549941480997, 0.9175578937577424)
clustering(2.497549941480997, 0.9175578937577424) finished, time: 288.25104427337646
started clustering(2.497549941480997, 0.9175578937577424)


[I 2022-05-04 08:32:36,685] Trial 0 finished with value: 0.2531309811712134 and parameters: {'eps': 2.497549941480997, 'q': 0.9175578937577424}. Best is trial 0 with value: 0.2531309811712134.


clustering(2.497549941480997, 0.9175578937577424) finished, time: 283.33678579330444
started clustering(2.4764710398009884, 0.6258603748577426)
clustering(2.4764710398009884, 0.6258603748577426) finished, time: 279.15098214149475
started clustering(2.4764710398009884, 0.6258603748577426)


[I 2022-05-04 08:41:55,662] Trial 1 finished with value: 0.5884238430654121 and parameters: {'eps': 2.4764710398009884, 'q': 0.6258603748577426}. Best is trial 1 with value: 0.5884238430654121.


clustering(2.4764710398009884, 0.6258603748577426) finished, time: 279.5311133861542
started clustering(2.304903966770585, 0.35711452666154797)
clustering(2.304903966770585, 0.35711452666154797) finished, time: 282.3912651538849
started clustering(2.304903966770585, 0.35711452666154797)


[I 2022-05-04 08:51:15,751] Trial 2 finished with value: 0.4761553117193672 and parameters: {'eps': 2.304903966770585, 'q': 0.35711452666154797}. Best is trial 1 with value: 0.5884238430654121.


clustering(2.304903966770585, 0.35711452666154797) finished, time: 277.5604603290558
started clustering(2.090102233707356, 0.4604576049676584)
clustering(2.090102233707356, 0.4604576049676584) finished, time: 282.268919467926
started clustering(2.090102233707356, 0.4604576049676584)


[I 2022-05-04 09:00:38,991] Trial 3 finished with value: 0.8295096484462443 and parameters: {'eps': 2.090102233707356, 'q': 0.4604576049676584}. Best is trial 3 with value: 0.8295096484462443.


clustering(2.090102233707356, 0.4604576049676584) finished, time: 280.8363630771637


In [41]:
#from scipy.optimize import shgo, dual_annealing, differential_evolution, basinhopping

#for f in [dual_annealing, differential_evolution]:
#    res = f(lambda eps: -objective(eps), [(1.5, 3.5)], maxiter=100)
#    print(res)


In [42]:
check_clustering(best_eps, best_q)


started clustering(2.090102233707356, 0.4604576049676584)
clustering(2.090102233707356, 0.4604576049676584) finished, time: 277.0366246700287
correct clustering
nr of classes: 230




found clustering
eps: 2.090102233707356
q: 0.4604576049676584
ACCURACY: 0.9807947989597919
BALANCE ACCURACY: 0.8203232791384008
started clustering(2.090102233707356, 0.4604576049676584)
clustering(2.090102233707356, 0.4604576049676584) finished, time: 310.6384320259094
started clustering(2.090102233707356, 0.4604576049676584)
clustering(2.090102233707356, 0.4604576049676584) finished, time: 368.28652334213257
AVERAGE BALANCED ACCURACY: 0.8286446300116692
nr of classes: 638


In [42]:
#print(initial_df)
def get_best_params():
    return get_best_params_optuna(TIMEOUT)

best_eps, best_q = get_best_params()


In [ ]:
#from scipy.optimize import shgo, dual_annealing, differential_evolution, basinhopping

#for f in [dual_annealing, differential_evolution]:
#    res = f(lambda eps: -objective(eps), [(1.5, 3.5)], maxiter=100)
#    print(res)


In [ ]:
check_clustering(best_eps, best_q)
